<a href="https://colab.research.google.com/github/tselane2110/ML-Specialization/blob/main/Logistic_Regression_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Logistic Regression without Regularization

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import math
class LogisticRegression:
  def fit_data(self, x, y):
    """
    self.x -> training data (features)
    self.y -> training data (labels/target-values/actual-values)
    self.m -> no of training examples
    self.n -> no of features
    self.w -> initializing weights
    self.b -> initializing bias
    self.y_hat -> initializing prediction's list
    self.cost_rec -> record of cost function of all iterations
    self.min_iterations -> minimum number of iterations for our gradient descent
    self.prediction_threshold -> if the output of sigmoid function is greater than or equal to this threshold, then y_hat will be 1, else 0
    """
    self.x = x
    self.y = y
    self.m = len(self.x)
    self.n = self.x.shape[1]
    self.w = np.zeros(self.n)
    self.b = 0
    self.y_hat = []
    self.cost_rec = []
    self.min_iterations = 40
    self.prediction_threshold = 0.5

    # calling recursive gradient descent
    self.recursiveGD()

  def predict_training(self, x):
    y_hat = []
    z = []
    e = math.e
    for i in range(len(x)):
      z.append(np.dot(self.w, x[i]) + self.b)
    for i in range(len(z)):
      y_hat.append((1/(1+(pow(e, -(z[i]))))))
    return y_hat

  def predict(self, x):
    y_hat = []
    z = []
    e = math.e
    for i in range(len(x)):
      z.append(np.dot(self.w, x[i]) + self.b)
    for i in range(len(z)):
      y_hat.append((1/(1+(pow(e, -(z[i]))))))
    for i in range(len(y_hat)):
      if y_hat[i] >= self.prediction_threshold:
        y_hat[i] = 1
      else:
        y_hat[i] = 0
    return y_hat

  def calculate_loss(self, y_hat, y):
    loss = []
    for i in range(len(y_hat)):
      a = -y[i] * math.log(y_hat[i])
      b = (1 - y[i]) * math.log(1 - y_hat[i])
      loss.append(a-b)
    return loss
    """
    loss = []
    for i in range(len(y_hat)):
      if y[i] == 1:
        loss.append(-math.log(y_hat[i]))
      else:
        loss.append(-math.log(1 - y_hat[i]))
    return loss"""

  def calculate_cost(self, loss):
    cost = 0
    for i in range(len(loss)):
      cost += loss[i]
    cost += (1/self.m)
    return cost

  def abs_diff(self, a, b):
    # abs_diff stands for absolute difference
    result = abs(a-b)
    return result

  def stopping_criteria_check(self, threshold):
    '''
    Need to check that the cost function for the last 20 iterations has converged
    Also verifying that we have calculated the cost atleast 40 times, idk for what reason but yea (I do know, i mean that's the min number
    of iterations Im aiming for)
    '''
    cost = self.cost_rec

    if len(cost) >= self.min_iterations :
      for i in range(-21, -1, 1):
        if self.abs_diff(cost[i], cost[i+1]) > threshold:
          return 1
        else:
          return 0

    else:
      return 1

  def plot_learningCurve(self):
    # printing the total number of iterations
    print("total number of iterations: ", len(self.cost_rec))

    # plotting the graph
    plt.plot(range(len(self.cost_rec)), self.cost_rec, label='Learning Curve')  # Plot cost_function vs # of iterations
    plt.xlabel('Number of Iterations')
    plt.ylabel('Cost Function J(w, b)')
    plt.title('Learning Curve ')
    plt.legend()
    plt.show()

  def calculate_error_w(self, j):
    error_wj = 0
    for i in range(len(self.y_hat)):
      error_wj += (self.y_hat[i] - self.y[i]) * self.x[i][j]
    return error_wj

  def calculate_error_b(self):
    error_b = 0
    for i in range(len(self.y_hat)):
      error_b += (self.y_hat[i] - self.y[i])
    return error_b

  def recursiveGD(self):
    # making predictions
    self.y_hat = self.predict_training(self.x)

    # calculate loss
    loss = self.calculate_loss(self.y_hat, self.y)

    # caculate cost
    self.cost_rec.append(self.calculate_cost(loss))

    # calling recursive gradient descent
    self.gradient_descent()


  def gradient_descent(self):
    alpha = 0.01
    threshold = 1e-3 # 0.001

    # stopping criteria
    sc = self.stopping_criteria_check(threshold)
    if sc == 0:
      self.plot_learningCurve()
      return

    # updating w
    for j in range(len(self.w)):
      error = self.calculate_error_w(j)
      self.w[j] = self.w[j] - (alpha * (1/self.m) * error)

    # updating b
    error = self.calculate_error_b()
    self.b = self.b - (alpha * (1/self.m) * error)

    # calling recursive gradient descent
    self.recursiveGD()



### Importing Dataset to test the model

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split

# change the max number of rows/columns to be displayed in the cell
pd.set_option("display.max_rows", 500)

# importing dataset
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MachineLearningSpecialization_AndrewNg/Telco-Customer-Churn.csv")

# encoding
df[df.columns]=df[df.columns].apply(lambda col:pd.Categorical(col).codes)

# dropping extra columns
df=df.drop(columns=["gender","TotalCharges", "StreamingMovies", "StreamingTV", "InternetService", "MultipleLines",\
                    "PhoneService", "MonthlyCharges"])

# distributing the data into independent and dependent variables:
x=df[['SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'Contract', 'PaperlessBilling', 'PaymentMethod']]
y=df["Churn"]

# splitting data into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [38]:
x_train.shape, y_train.shape

((4930, 11), (4930,))

# Testing our Logistic Regression model without regularization

In [39]:
model = LogisticRegression()

In [40]:
model.fit_data(np.array(x_train), np.array(y_train))

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
y_hat = model.predict(x_test)
error = y_hat - y_test
error

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix= pd.crosstab(y_test,y_hat, rownames=['actual'],colnames=['predicted'])
accuracy_logistic=accuracy_score(y_test, y_hat)
print(confusion_matrix, accuracy_logistic)